Code to test power_flow_processing library

In [1]:
import power_flow_processing as pfp
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file
import numpy as np

In [2]:
# with open("input_network_data.json") as fp:
#     data = fp.read()
# pprint.pprint(json.loads(data))

dataset = json_deserialize_from_file("input_network_data.json")

print(type(dataset))
print(pd.DataFrame(dataset.keys()))

print(pd.DataFrame(dataset['line']))
print(pd.DataFrame(dataset['node']))
print(pd.DataFrame(dataset['source']))
print(pd.DataFrame(dataset['sym_load']))

<class 'dict'>
          0
0      node
1      line
2    source
3  sym_load
   id  from_node  to_node  from_status  to_status    r1   x1       c1  tan1  \
0   5          1        2            1          1  0.25  0.2  0.00001   0.0   
1   6          2        3            1          1  0.25  0.2  0.00001   0.0   
2   7          3        4            1          1  0.25  0.2  0.00001   0.0   

   r0  x0  c0  tan0     i_n  
0 NaN NaN NaN   NaN  1000.0  
1 NaN NaN NaN   NaN  1000.0  
2 NaN NaN NaN   NaN  1000.0  
   id  u_rated
0   1  10500.0
1   2  10500.0
2   3  10500.0
3   4  10500.0
   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  11     1       1    1.0          NaN  200000000.0       NaN        NaN
   id  node  status  type  p_specified  q_specified
0   8     2       1     0          0.0          0.0
1   9     3       1     0          0.0          0.0
2  10     4       1     0          0.0          0.0


In [3]:
active_power_data = pd.read_parquet('active_power_profile.parquet')
print(type(active_power_data))

print(active_power_data.index.tolist())

<class 'pandas.core.frame.DataFrame'>
[Timestamp('2024-01-01 00:00:00'), Timestamp('2024-01-01 01:00:00'), Timestamp('2024-01-01 02:00:00'), Timestamp('2024-01-01 03:00:00'), Timestamp('2024-01-01 04:00:00'), Timestamp('2024-01-01 05:00:00'), Timestamp('2024-01-01 06:00:00'), Timestamp('2024-01-01 07:00:00'), Timestamp('2024-01-01 08:00:00'), Timestamp('2024-01-01 09:00:00')]


In [4]:
reactive_power_data = pd.read_parquet('active_power_profile.parquet')
print(type(reactive_power_data))

print(reactive_power_data)


<class 'pandas.core.frame.DataFrame'>
Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00   97627.007855  430378.732745  205526.752143
2024-01-01 01:00:00   89766.365994 -152690.401322  291788.226133
2024-01-01 02:00:00 -124825.577475  783546.001564  927325.521002
2024-01-01 03:00:00 -233116.962348  583450.076165   57789.839506
2024-01-01 04:00:00  136089.122188  851193.276585 -857927.883604
2024-01-01 05:00:00 -825741.400597 -959563.205119  665239.691096
2024-01-01 06:00:00  556313.501900  740024.296494  957236.684466
2024-01-01 07:00:00  598317.128433  -77041.275494  561058.352573
2024-01-01 08:00:00 -763451.148262  279842.042655 -713293.425182
2024-01-01 09:00:00  889337.834099   43696.643500 -170676.120019


In [5]:
grid1 = pfp.PowerFlow(grid_data=dataset)

output_data = grid1.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

print(grid1.aggregate_voltage_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data))


            Timestamp  max_id  u_pu_max  min_id  u_pu_min
0 2024-01-01 00:00:00       1  1.001126       4  0.997504
1 2024-01-01 01:00:00       3  1.004964       1  1.003940
2 2024-01-01 02:00:00       1  0.996233       4  0.981468
3 2024-01-01 03:00:00       1  1.002937       3  1.001172
4 2024-01-01 04:00:00       4  1.010267       1  1.004481
5 2024-01-01 05:00:00       3  1.019485       1  1.011274
6 2024-01-01 06:00:00       1  0.992400       4  0.974659
7 2024-01-01 07:00:00       1  0.999169       4  0.993243
8 2024-01-01 08:00:00       4  1.023922       1  1.011689
9 2024-01-01 09:00:00       4  1.001906       2  0.999437


In [6]:
voltage_table = pd.DataFrame()

voltage_table["Timestamp"] = active_power_data.index.tolist()
voltage_table["max_id"] = output_data["node"][:, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmax(axis=1).tolist() ]["id"][0, :]
voltage_table["u_pu_max"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).max(axis=1).tolist()
voltage_table["min_id"] = output_data["node"][:, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmin(axis=1).tolist() ]["id"][0, :]
voltage_table["u_pu_min"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).min(axis=1).tolist()



print(pd.DataFrame(voltage_table))

            Timestamp  max_id  u_pu_max  min_id  u_pu_min
0 2024-01-01 00:00:00       1  1.001126       4  0.997504
1 2024-01-01 01:00:00       3  1.004964       1  1.003940
2 2024-01-01 02:00:00       1  0.996233       4  0.981468
3 2024-01-01 03:00:00       1  1.002937       3  1.001172
4 2024-01-01 04:00:00       4  1.010267       1  1.004481
5 2024-01-01 05:00:00       3  1.019485       1  1.011274
6 2024-01-01 06:00:00       1  0.992400       4  0.974659
7 2024-01-01 07:00:00       1  0.999169       4  0.993243
8 2024-01-01 08:00:00       4  1.023922       1  1.011689
9 2024-01-01 09:00:00       4  1.001906       2  0.999437


Error testing

In [7]:
from power_grid_model import PowerGridModel, initialize_array
import power_flow_processing as pfp

# A power grid containing several errors

# node
node_error = initialize_array("input", "node", 3)
node_error["id"] = [1, 2, 3]
node_error["u_rated"] = [10.5e3]

# line
line_error = initialize_array("input", "line", 3)
line_error["id"] = [4, 5, 6]
line_error["from_node"] = [1, 2, 3]
line_error["to_node"] = [2, 3, 4]
line_error["from_status"] = [True]
line_error["to_status"] = [True]
line_error["r1"] = [0.25]
line_error["x1"] = [0.2]
line_error["c1"] = [10e-6]
line_error["tan1"] = [0.0]

# Power Sensor
sensor_error = initialize_array("input", "sym_power_sensor", 2)
sensor_error["id"] = [6, 7]
sensor_error["measured_object"] = [3, 4]
sensor_error["measured_terminal_type"] = [0, 2]
sensor_error["p_measured"] = [0]
sensor_error["q_measured"] = [0]
sensor_error["power_sigma"] = [0]

error_data = {"node": node_error, "line": line_error, "sym_power_sensor": sensor_error}

pfp.PowerFlow(grid_data=error_data)

ValidationException: There are 2 validation errors in input_data:
   1. Fields line.id and sym_power_sensor.id are not unique for 2 lines/sym_power_sensors.
   2. Field 'to_node' does not contain a valid node id for 1 line.